# **Hotel Booking Demand Dataset**

In [ ]:
#Google Colab link: https://colab.research.google.com/drive/1F8b1h9QDBvPKKIT2wYTY6Vdhx7Rs_HiF?usp=sharing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import folium

from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

sns.set(style="whitegrid")
pd.set_option("display.max_columns", 36)

In [ ]:
# Load dataset
df = pd.read_csv('hotel_bookings.csv', engine='python')
df.shape
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['meal'].unique()

In [ ]:
df['country'].unique()

# **Part B: Data Cleaning**

a. Handling Missing Values

b. Handling Noise

c. Handling inconsistent data



In [ ]:
clean_df = pd.read_csv('hotel_bookings.csv', engine='python')
clean_df.shape
clean_df

In [ ]:
#1. Check for missing values
clean_df.isnull().sum()

In [ ]:
#2. Check datatype
clean_df.dtypes

In [ ]:
clean_df = df
clean_df.head(5)

In [ ]:
print("Shape before Data Cleaning :", end=" ")
print(clean_df.shape)
print ("")

In [ ]:
#To remove rows where children, adults and babies are 0 at the same time
clean_df = clean_df.drop(clean_df[(clean_df.adults+clean_df.babies+clean_df.children)==0].index)

#To drop rows with null values in column 'children'. 
#(since number of missing value is 4, it is insignificant, hence we drop it)
clean_df = clean_df.dropna(axis=0, subset=['children'])

#To replace null values in column 'country' with 'N.A'
clean_df['country'] = clean_df['country'].fillna("N.A")

#To replace 'meal - Undefined' to 'SC'.
#clean_df['meal'] = clean_df["meal"].replace("Undefined", "SC")

#To delete column 'company' from dataset because it has the most number of missing values.
#Data is given in company ID, thus insignificant in project as we do not know the specific company names
clean_df.drop(columns=["company"], inplace=True)

#To drop column 'agent' for the same reason as company
clean_df.drop(columns=["agent"], inplace=True)

#To drop unwanted columns
#clean_df.drop(['reservation_status_date'], axis = 1, inplace = True)

In [ ]:
print("Shape after Data Cleaning :", end=" ")
print(clean_df.shape)

In [ ]:
clean_df

In [ ]:
print("Shape before Data Cleaning :", end=" ")
print(clean_df.shape)
print ("")

#Abnormal Values

#1. To drop rows with 0 adults. (Adult needed to book a room)
clean_df.drop(labels = clean_df[clean_df['adults'] == 0].index, axis = 0, inplace = True)
clean_df[clean_df['adults'] == 0]

#2. -ve values of ADR
clean_df = clean_df.drop(clean_df[clean_df.adr < 0].index)
clean_df[clean_df['adr'] < 0]

print("Shape after Data Cleaning :", end=" ")
print(clean_df.shape)

In [ ]:
#Further data cleaning

#To get total number of kids (reduce number of columns)
clean_df['kids'] = clean_df.children + clean_df.babies

In [ ]:
#Then, removing adults, children and babies column
#Reason: Because it is insignificant to our business question and goal
clean_df.drop(['children','babies'], axis = 1, inplace = True)

clean_df.head(5)

In [ ]:
#To check again for missing values
clean_df.isnull().sum()

In [ ]:
clean_df.head(5)

In [ ]:
clean_df.describe()

In [ ]:
columns = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'required_car_parking_spaces', 'adr', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes']
n = 1
plt.figure(figsize=(20,15))

for column in columns:
  plt.subplot(4,4,n)
  n = n+1
  sns.boxplot(df[column])
  plt.tight_layout()

In [ ]:
#remove the outliers by using conditionals to get and update these values.

clean_df.loc[clean_df.lead_time > 500, 'lead_time'] = 500
clean_df.loc[clean_df.stays_in_weekend_nights >=  5, 'stays_in_weekend_nights'] = 5
clean_df.loc[clean_df.adults > 4, 'adults'] = 4
clean_df.loc[clean_df.previous_bookings_not_canceled > 0, 'previous_bookings_not_canceled'] = 1
clean_df.loc[clean_df.previous_cancellations > 0, 'previous_cancellations'] = 1
clean_df.loc[clean_df.stays_in_week_nights > 10, 'stays_in_week_nights'] = 10
clean_df.loc[clean_df.booking_changes > 5, 'booking_changes'] = 5
clean_df.loc[clean_df.required_car_parking_spaces > 5, 'required_car_parking_spaces'] = 0
clean_df.loc[clean_df.adr > 1000, 'adr'] = 1000

In [ ]:
clean_df.describe()

In [ ]:
# clean_df.to_csv('hotel_bookings_cleaned.csv')

In [ ]:
clean_df.head(5)

In [ ]:
correlation = clean_df.corr()['is_canceled'].abs().sort_values(ascending = False)
correlation

In [ ]:
# dropping columns that are not useful

useless_col = ['days_in_waiting_list', 'arrival_date_year', 'arrival_date_year', 'assigned_room_type', 'booking_changes',
               'reservation_status', 'country', 'days_in_waiting_list']

clean_df.drop(useless_col, axis = 1, inplace = True)

In [ ]:
clean_df.head()

In [ ]:
# creating numerical and categorical dataframes

cat_cols = [col for col in clean_df.columns if clean_df[col].dtype == 'O']
cat_cols

In [ ]:
cat_df = clean_df[cat_cols]
cat_df.head()

In [ ]:
cat_df['reservation_status_date'] = pd.to_datetime(cat_df['reservation_status_date'])
cat_df['year'] = cat_df['reservation_status_date'].dt.year
cat_df['month'] = cat_df['reservation_status_date'].dt.month
cat_df['day'] = cat_df['reservation_status_date'].dt.day

In [ ]:
cat_df.drop(['reservation_status_date','arrival_date_month'] , axis = 1, inplace = True)

In [ ]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

# **Part C: Data Reduction**

### **Dimensionality Reduction (data encoding)**

In [ ]:
# encoding categorical variables

cat_df['hotel'] = cat_df['hotel'].map({'Resort Hotel' : 0, 'City Hotel' : 1})

cat_df['meal'] = cat_df['meal'].map({'BB' : 0, 'FB': 1, 'HB': 2, 'SC': 3, 'Undefined': 4})

cat_df['market_segment'] = cat_df['market_segment'].map({'Direct': 0, 'Corporate': 1, 'Online TA': 2, 'Offline TA/TO': 3,
                                                           'Complementary': 4, 'Groups': 5, 'Undefined': 6, 'Aviation': 7})

cat_df['distribution_channel'] = cat_df['distribution_channel'].map({'Direct': 0, 'Corporate': 1, 'TA/TO': 2, 'Undefined': 3,
                                                                       'GDS': 4})

cat_df['reserved_room_type'] = cat_df['reserved_room_type'].map({'C': 0, 'A': 1, 'D': 2, 'E': 3, 'G': 4, 'F': 5, 'H': 6,
                                                                   'L': 7, 'B': 8})

cat_df['deposit_type'] = cat_df['deposit_type'].map({'No Deposit': 0, 'Refundable': 1, 'Non Refund': 3})

cat_df['customer_type'] = cat_df['customer_type'].map({'Transient': 0, 'Contract': 1, 'Transient-Party': 2, 'Group': 3})

cat_df['year'] = cat_df['year'].map({2015: 0, 2014: 1, 2016: 2, 2017: 3})

In [ ]:
cat_df.head()

In [ ]:
num_df = clean_df.drop(columns = cat_cols, axis = 1)
#num_df.drop('is_canceled', axis = 1, inplace = True)
num_df

In [ ]:
num_df.var()

### **Perform Data Tranformation**

In [ ]:
# normalizing numerical variables

num_df['lead_time'] = np.log(num_df['lead_time'] + 1)
num_df['arrival_date_week_number'] = np.log(num_df['arrival_date_week_number'] + 1)
num_df['arrival_date_day_of_month'] = np.log(num_df['arrival_date_day_of_month'] + 1)
num_df['adr'] = np.log(num_df['adr'] + 1)

In [ ]:
num_df.var()

In [ ]:
num_df['adr'] = num_df['adr'].fillna(value = num_df['adr'].mean())

In [ ]:
num_df.head()

In [ ]:
df_new = pd.concat([cat_df, num_df], axis=1, join='inner')
df_new

In [ ]:
df_new.corr()['is_canceled'].abs().sort_values(ascending = False)

### **Perform Dimensionality Reduction using PCA**

In [ ]:
variables = ['lead_time', 'total_of_special_requests', 'market_segment', 
             'deposit_type', 'previous_cancellations', 'is_canceled']

df_new[variables[:-1]]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca

In [ ]:
transformed = pca.fit_transform(df_new[variables[:-1]])
transformed

In [ ]:
len(pca.components_)

In [ ]:
sum(pca.explained_variance_ratio_[0:1])

In [ ]:
dataset_pca = pd.DataFrame(transformed)
dataset_pca.drop([1, 2, 3, 4], axis = 1, inplace = True)
dataset_pca

In [ ]:
dataset_pca['is_canceled'] = df_new['is_canceled']
dataset_pca.is_canceled

In [ ]:
dataset_pca.columns = ['pc1', 'is_canceled']

dataset_pca

In [ ]:
print(f'df_new -> {df_new[variables].shape}\ndataset_pca -> {dataset_pca.shape}')

In [ ]:
variables = list(dataset_pca.columns)
variables

# **Part D: Results Presentation and Visualisation**



Labelling of hotel, market_segment, distribution_channel. (before)

In [ ]:
labels = df['hotel'].value_counts().index.tolist()
sizes = df['hotel'].value_counts().tolist()
explode = (0, 0.05)
colors = ['yellow', 'orange']

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',startangle =90, textprops={'fontsize': 14})
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
sns.countplot(x='market_segment', data=df, palette='rocket')
plt.title('Types of market segment',fontweight="bold", size=20)

plt.subplot(1, 2, 2)
sns.countplot(data = df, x = 'distribution_channel',  palette='Set1_r')
plt.title('Types of distribution channels',fontweight="bold", size=20)
plt.subplots_adjust(right=1.7)


plt.show()

Labelling of hotel, market_segment, distribution_channel. (after)

In [ ]:
labels = df_new['hotel'].value_counts().index.tolist()
sizes = df_new['hotel'].value_counts().tolist()
explode = (0, 0.05)
colors = ['orangered', 'blue']

plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',startangle =90, textprops={'fontsize': 14})
plt.legend(['City Hotel', 'Resort Hotel'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.subplot(1, 2, 1)
sns.countplot(x='market_segment', data=df_new, palette='rocket')
plt.title('Types of market segment',fontweight="bold", size=20)
plt.legend(['Direct', 'Corporate', 'Online TA', 'Offline TA/TO', 'Complementary', 'Groups', 'Undefined', 'Aviation'], loc='upper right')

plt.subplot(1, 2, 2)
sns.countplot(data = df_new, x = 'distribution_channel',  palette='Set1_r')
plt.title('Types of distribution channels',fontweight="bold", size=20)
plt.subplots_adjust(right=1.7)
plt.legend(['Direct', 'Corporate', 'TA/TO', 'Undefined', 'GDS'], loc='upper right')

plt.show()

Outliers of data. (before)

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(df['stays_in_weekend_nights'],label="Weekend nights stays",axlabel=False, kde=False)
sns.distplot(df['stays_in_week_nights'],label="Week nights stays",axlabel=False, kde=False)
plt.legend()

Outliers removed by using conditionals statement. (after)

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(df_new['stays_in_weekend_nights'],label="Weekend nights stays",axlabel=False, kde=False)
sns.distplot(df_new['stays_in_week_nights'],label="Week nights stays",axlabel=False, kde=False)
plt.legend()

Columns children & babies before data reduction.

In [ ]:
sns.countplot(data = df, x = 'children', hue = 'hotel').set_title("Number of children", fontsize = 20)
plt.legend(loc='upper right')

In [ ]:
sns.countplot(data = df, x = 'babies', hue = 'hotel').set_title("Number of babies", fontsize = 20)
plt.legend(loc='upper right')

Combine children and babies together as kids, then columns children & babies removed after data reduction.

In [ ]:
sns.countplot(data = df_new, x = 'kids', hue = 'hotel').set_title("Number of kids", fontsize = 20)
plt.legend(['Resort Hotel', 'City Hotel'], loc='upper right')

In [ ]:
df_new.to_csv('hotel_bookings_cleaned.csv')

In [ ]:
dataset_pca.to_csv('hotel_bookings_cleaned1.csv')